In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [24]:
def get_jobs(keyword, num_jobs, verbose, path):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)

    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector("[alt='Close']").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [26]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 10, False, "/Users/Dev/Desktop/Data Science Projects/Salary Prediction Glassdoor/chromedriver")
df

Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Junior Data Scientist,$81K-$134K (Glassdoor est.),"Company\n\nFundation is a high growth, private...",4.5,Fundation Group LLC.\n4.5,"Reston, VA",-1,51 to 200 Employees,2011,Company - Private,Lending,Finance,$50 to $100 million (USD),-1
1,"Data Scientist, Health Economics & Advanced An...",$81K-$134K (Glassdoor est.),Overview\n\n\nAvalere Health is seeking a Data...,3.2,Avalere Health LLC\n3.2,"Washington, DC",-1,201 to 500 Employees,2000,Company - Public,Consulting,Business Services,Less than $1 million (USD),-1
2,Research Scientist II Formulation,$81K-$134K (Glassdoor est.),The Research Scientist II Formulation develops...,3.1,Amneal Pharmaceuticals L.L.C.\n3.1,"Brookhaven, NY",-1,5001 to 10000 Employees,2002,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$1 to $2 billion (USD),-1
3,Research Scientist,$81K-$134K (Glassdoor est.),Enabling Immunotherapy*\nKey responsibilities ...,-1,"immunoSCAPE, Inc.","San Diego, CA",-1,-1,-1,-1,-1,-1,-1,-1
4,Scientist,$81K-$134K (Glassdoor est.),Description:\n\nJOB SUMMARY\n\nContribute to t...,3.5,Grand River Aseptic Manufacturing\n3.5,"Grand Rapids, MI",-1,51 to 200 Employees,-1,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Less than $1 million (USD),-1
5,"Senior Data Scientist, Zoro",$81K-$134K (Glassdoor est.),Company Summary:\nZoro.com is an eCommerce com...,2.0,Zoro Tools Europe GmbH\n2.0,"Chicago, IL",-1,Unknown,-1,Company - Private,-1,-1,Less than $1 million (USD),-1
6,Bioinformatics Scientist,$81K-$134K (Glassdoor est.),Job Description: *\nThe successful candidate w...,-1,OME Wellness,"San Diego, CA",-1,-1,-1,-1,-1,-1,-1,-1
7,SR BIOINFORMATICS SCIENTIST OB/GYN RESEARCH,$81K-$134K (Glassdoor est.),Senior Bioinformatics Scientist OB/GYN Researc...,3.3,Inova\n3.3,"Falls Church, VA",-1,10000+ Employees,1956,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$10 to $25 million (USD),-1
8,Data Scientist,$81K-$134K (Glassdoor est.),Job Description:\n\nMetron is seeking talented...,3.0,"Metron, Inc.\n3.0","Reston, VA",-1,51 to 200 Employees,-1,Company - Private,Electrical & Electronic Manufacturing,Manufacturing,$25 to $50 million (USD),-1
9,Data Engineer,$81K-$134K (Glassdoor est.),Get in on the ground floor as part of an excit...,3.0,Perimetral Montagens\n3.0,"Los Angeles, CA",-1,Unknown,-1,Company - Private,-1,-1,Less than $1 million (USD),-1
